In [ ]:
!pip in!apt-get update
!apt-get install -y tesseract-ocr
!pip install pytesseract
!pip install ultralytics huggingface_hub supervision


In [ ]:
import os
import cv2
import numpy as np
import pytesseract
from pytesseract import Output
from ultralytics import YOLO

# Create a folder for saving cropped faces
os.makedirs("results", exist_ok=True)

# Load the YOLO face detection model
model = YOLO("yolov10n-face.pt")  # Use your pretrained YOLO face model

# List of image filenames to process (Modify as needed)
filenames = #include the path of the image

# Confidence threshold (75%)
CONFIDENCE_THRESHOLD = 0.70

# Function to detect text orientation using Tesseract
def detect_text_orientation(image):
    osd = pytesseract.image_to_osd(image, output_type=Output.DICT)
    return osd.get("rotate", 0)

# Function to rotate the image based on detected text orientation
def rotate_image(image, angle):
    if angle == 0:
        return image
    elif angle == 90:
        return cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
    elif angle == 180:
        return cv2.rotate(image, cv2.ROTATE_180)
    elif angle == 270:
        return cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE)
    else:
        raise ValueError(f"Unsupported rotation angle: {angle}")

# Process each image in batch
for i, filename in enumerate(filenames):
    # Load the image
    img = cv2.imread(filename)
    if img is None:
        print(f"Could not load image: {filename}")
        continue

    # Detect and correct text orientation
    rotation_angle = detect_text_orientation(img)
    print(f"Image {filename}: Detected rotation angle = {rotation_angle}°")
    img = rotate_image(img, rotation_angle)

    # Detect faces using YOLO
    results = model(img)

    # Process each detected face
    for j, result in enumerate(results):
        # Convert bounding boxes and confidence scores to numpy arrays
        boxes = result.boxes.xyxy.cpu().numpy()
        confidences = result.boxes.conf.cpu().numpy()

        for k, (box, conf) in enumerate(zip(boxes, confidences)):
            if conf < CONFIDENCE_THRESHOLD:
                print(f"Skipping face {k} in {filename} (Confidence: {conf:.2f})")
                continue  # Ignore low-confidence detections

            x1, y1, x2, y2 = map(int, box)  # Extract box coordinates

            # Compute center, width, and height of original bounding box
            cx, cy = (x1 + x2) / 2, (y1 + y2) / 2
            w, h = (x2 - x1), (y2 - y1)

            # Increase bounding box size by 2x
            new_w, new_h = w * 2, h * 2

            # Compute new coordinates ensuring they stay within image boundaries
            new_x1 = int(max(cx - new_w / 2, 0))
            new_y1 = int(max(cy - new_h / 2, 0))
            new_x2 = int(min(cx + new_w / 2, img.shape[1] - 1))
            new_y2 = int(min(cy + new_h / 2, img.shape[0] - 1))

            # Crop the enlarged bounding box region
            face_crop = img[new_y1:new_y2, new_x1:new_x2]

            # Save cropped face
            crop_filename = os.path.join("results", f"crop_{i}_{k}.jpg")
            cv2.imwrite(crop_filename, face_crop)
            print(f"Saved cropped face: {crop_filename} (Confidence: {conf:.2f})")
